In [6]:
import pickle
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import pandas as pd
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import joblib

In [8]:
import seaborn as sns
import collections

In [9]:
df = joblib.load("texts_orinal_data.pkl")
df = df[["caption","caption_pos","caption_neg"]]
def make_l(v):
    return len(v)
for col in ["caption","caption_pos","caption_neg"]:
    df[col+"_L"] = df[col].apply(make_l)

In [10]:
df["caption_L"].describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])

count    122810.000000
mean         21.299919
std           5.931135
min           6.000000
10%          15.000000
20%          17.000000
30%          18.000000
40%          19.000000
50%          20.000000
60%          21.000000
70%          23.000000
80%          25.000000
90%          29.000000
100%        119.000000
max         119.000000
Name: caption_L, dtype: float64

In [11]:
df["caption_pos_L"].describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])

count    122810.000000
mean         21.295155
std           5.989772
min           7.000000
10%          15.000000
20%          17.000000
30%          18.000000
40%          19.000000
50%          20.000000
60%          21.000000
70%          23.000000
80%          25.000000
90%          29.000000
100%        108.000000
max         108.000000
Name: caption_pos_L, dtype: float64

In [12]:
df["caption_neg_L"].describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])

count    122810.000000
mean         20.949043
std           5.812047
min           2.000000
10%          15.000000
20%          16.000000
30%          17.000000
40%          18.000000
50%          20.000000
60%          21.000000
70%          23.000000
80%          25.000000
90%          28.000000
100%        149.000000
max         149.000000
Name: caption_neg_L, dtype: float64

In [13]:
from transformers import DistilBertTokenizer
max_seq_length = 28
train_output_file = "texts_train_data.tfrecord"
test_output_file = "texts_test_data.tfrecord"
tokenizer = DistilBertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


In [16]:
def create_code(text1,text2,label_id,tf_record_writer):
    all_features = collections.OrderedDict()
    #encode_plus_tokens = tokenizer.encode_plus(f"{text1}[SEP]{text2}",padding='max_length',max_length=max_seq_length*2,truncation=True)
    encode_plus_tokens = tokenizer.encode_plus(text1,padding='max_length',max_length=max_seq_length,truncation=True)
    input_ids1 = encode_plus_tokens["input_ids"]
    input_mask1 = encode_plus_tokens["attention_mask"]
    segment_ids1 = [0] *max_seq_length
    
    encode_plus_tokens = tokenizer.encode_plus(text2,padding='max_length',max_length=max_seq_length,truncation=True)
    input_ids2 = encode_plus_tokens["input_ids"]
    input_mask2 = encode_plus_tokens["attention_mask"]
    segment_ids2 = [0] *max_seq_length

    #label_id = text = dt["Y"][i]
    all_features["input_ids1"] = tf.train.Feature(int64_list=tf.train.Int64List(value=input_ids1))
    all_features["input_mask1"] = tf.train.Feature(int64_list=tf.train.Int64List(value=input_mask1))
    all_features["segment_ids1"] = tf.train.Feature(int64_list=tf.train.Int64List(value=segment_ids1))
    all_features["input_ids2"] = tf.train.Feature(int64_list=tf.train.Int64List(value=input_ids2))
    all_features["input_mask2"] = tf.train.Feature(int64_list=tf.train.Int64List(value=input_mask2))
    all_features["segment_ids2"] = tf.train.Feature(int64_list=tf.train.Int64List(value=segment_ids2))
    all_features["label_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[label_id]))
    tf_record = tf.train.Example(features=tf.train.Features(feature=all_features))
    tf_record_writer.write(tf_record.SerializeToString())
def create_tf_data_file(dt,output_file):
    tf_record_writer = tf.io.TFRecordWriter(output_file)
    for i in range(dt.shape[0]):
        #label_id = i%2
        text1 = dt["caption"][i][0:max_seq_length]
        text2 = dt["caption_pos"][i][0:max_seq_length]
        text3 = dt["caption_neg"][i][0:max_seq_length]
        create_code(text1,text2,1,tf_record_writer)
        create_code(text1,text3,0,tf_record_writer)
    tf_record_writer.close()

In [17]:
from sklearn.model_selection import train_test_split
dt_train,dt_test = train_test_split(df,test_size=0.2)
dt_train = dt_train.reset_index()
dt_test = dt_test.reset_index()

In [18]:
create_tf_data_file(dt_train,train_output_file)
create_tf_data_file(dt_test,test_output_file)

In [29]:
9%2

1

In [12]:
text1 ='夜場所口'
text2 ='夜場所口'
tokenizer.encode_plus(f"{text1}[SEP]{text2}",padding='max_length',max_length=max_seq_length*2,truncation=True)

{'input_ids': [2, 1563, 173, 233, 1285, 3, 1563, 173, 233, 1285, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [20]:
len([2, 594, 335, 14, 98, 843, 291, 7, 287, 53, 581, 1582, 46, 28355, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

56

In [23]:
len([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

56

In [58]:
dt_train.shape

(98248, 7)

In [59]:
dt_test.shape

(24562, 7)